# <center> Обучение Catboost с фичами и эмбеддингами. </center>

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('sample_submit_naive.csv').drop('predict', axis=1)
test['target'] = -1

train = pd.concat([train, test])
train

,user_id,target,time
0,3,0,77.0
1,13,0,86.0
2,37,0,89.0
3,41,0,57.0
4,42,0,84.0
...,...,...,...
31995,561362,-1,NaN
31996,561419,-1,NaN
31997,561895,-1,NaN
31998,561908,-1,NaN


In [3]:
clients = pd.read_csv('clients.csv')
clients

,user_id,report,employee_count_nm,bankemplstatus,customer_age
0,3,2,ОТ 101 ДО 500,0,3
1,9,1,БОЛЕЕ 1001,0,3
2,13,6,ОТ 501 ДО 1000,0,2
3,37,5,БОЛЕЕ 1001,0,2
4,41,1,ОТ 101 ДО 500,0,2
...,...,...,...,...,...
95995,562043,12,NaN,0,2
95996,562205,12,NaN,0,1
95997,562312,12,NaN,0,0
95998,562721,12,NaN,0,2


In [4]:
train = train.merge(clients, on='user_id', how='left')
train

,user_id,target,time,report,employee_count_nm,bankemplstatus,customer_age
0,3,0,77.0,2,ОТ 101 ДО 500,0,3
1,13,0,86.0,6,ОТ 501 ДО 1000,0,2
2,37,0,89.0,5,БОЛЕЕ 1001,0,2
3,41,0,57.0,1,ОТ 101 ДО 500,0,2
4,42,0,84.0,12,ДО 10,0,3
...,...,...,...,...,...,...,...
95995,561362,-1,NaN,12,NaN,0,3
95996,561419,-1,NaN,12,NaN,0,3
95997,561895,-1,NaN,12,NaN,0,2
95998,561908,-1,NaN,12,NaN,0,2


In [5]:
cat_cols = ['report', 'employee_count_nm', 'bankemplstatus', 'customer_age']

## Загружаем полученные эмбеддинги и примерживаем к трейну

Можно скачать готовые [отсюда](https://www.kaggle.com/datasets/ivanblch/ptls-dtf/data)

In [6]:
ptls = pd.read_csv('ptls4.csv')
ptls.head()

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_119,embed_120,embed_121,embed_122,embed_123,embed_124,embed_125,embed_126,embed_127,user_id
0,-0.165534,0.351978,-0.350839,0.106836,0.293595,0.044469,0.414537,-0.166538,-0.351249,-0.310289,...,0.375856,0.090394,-0.129844,0.012284,0.578882,-0.004445,-0.658017,0.061427,0.675077,3
1,-0.063022,0.361478,-0.826556,-0.154087,0.046106,-0.072741,-0.035444,-0.654632,-0.685090,-0.128401,...,-0.448710,-0.232319,-0.431142,-0.145322,0.315182,-0.321269,-0.196726,-0.116659,0.896083,9
2,-0.246840,0.107814,0.341888,0.161511,0.088281,0.089849,0.682158,-0.471609,0.118671,-0.584602,...,0.036044,0.120221,-0.015794,-0.071148,0.504213,-0.205995,-0.490473,-0.324970,0.438195,13
3,-0.145627,0.025706,-0.542809,-0.197882,0.015115,-0.047794,0.429975,-0.059663,0.034127,0.237009,...,-0.432880,0.141057,-0.217942,0.160477,0.148012,0.019053,-0.345193,-0.098770,-0.708134,37
4,-0.083140,0.015028,-0.032059,0.202491,-0.145457,0.029561,-0.077098,-0.415457,-0.523322,-0.059340,...,-0.005664,0.235670,0.187565,-0.088271,-0.452672,-0.202955,-0.094149,-0.449056,0.548946,41


In [7]:
train_ptls = train.merge(ptls, on='user_id', how='left').fillna('nan')
train_ptls

,user_id,target,time,report,employee_count_nm,bankemplstatus,customer_age,embed_0,embed_1,embed_2,...,embed_118,embed_119,embed_120,embed_121,embed_122,embed_123,embed_124,embed_125,embed_126,embed_127
0,3,0,77.0,2,ОТ 101 ДО 500,0,3,-0.165534,0.351978,-0.350839,...,-0.382908,0.375856,0.090394,-0.129844,0.012284,0.578882,-0.004445,-0.658017,0.061427,0.675077
1,13,0,86.0,6,ОТ 501 ДО 1000,0,2,-0.246840,0.107814,0.341888,...,0.132429,0.036044,0.120221,-0.015794,-0.071148,0.504213,-0.205995,-0.490473,-0.324970,0.438195
2,37,0,89.0,5,БОЛЕЕ 1001,0,2,-0.145627,0.025706,-0.542809,...,0.522242,-0.432880,0.141057,-0.217942,0.160477,0.148012,0.019053,-0.345193,-0.098770,-0.708134
3,41,0,57.0,1,ОТ 101 ДО 500,0,2,-0.083140,0.015028,-0.032059,...,-0.192085,-0.005664,0.235670,0.187565,-0.088271,-0.452672,-0.202955,-0.094149,-0.449056,0.548946
4,42,0,84.0,12,ДО 10,0,3,-0.168607,-0.026094,0.153595,...,-0.743862,0.014378,0.451751,0.653084,0.231444,-0.321673,-0.293455,-0.156458,0.006764,0.958333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,561362,-1,nan,12,nan,0,3,0.058122,0.279635,-0.654057,...,-0.419299,-0.234023,0.120949,-0.525822,-0.009516,0.409767,0.171080,-0.372976,0.073020,-0.238165
95996,561419,-1,nan,12,nan,0,3,-0.151979,0.314061,-0.555999,...,-0.267269,-0.226834,0.237026,-0.338405,0.035742,-0.016611,-0.097133,0.135711,-0.141024,-0.457657
95997,561895,-1,nan,12,nan,0,2,0.125003,0.622484,-0.432142,...,0.510738,-0.271052,0.370545,-0.251802,0.251292,-0.502044,-0.049194,-0.400929,-0.207778,-0.760211
95998,561908,-1,nan,12,nan,0,2,0.157188,0.421542,-0.239896,...,-0.132614,-0.237024,0.303846,-0.004845,0.142564,-0.289598,-0.163889,0.012159,-0.167386,-0.530260


In [8]:
X = train_ptls[train_ptls.target != -1].drop(['user_id', 'target', 'time'], axis=1)
y = train_ptls[train_ptls.target != -1]['target']                        

In [9]:
n_splits=5
scores = []
models = []

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for train_index, val_index in skf.split(X, y):
    
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    

    model = CatBoostClassifier(random_seed=42,
                                eval_metric='AUC',
                                cat_features=cat_cols,
                                thread_count=-1,
                                #task_type='GPU',
    )

    model.fit(X_train, y_train,
              eval_set=(X_val, y_val),
              early_stopping_rounds=100,
              verbose=100)
    
    models.append(model)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    scores.append(roc_auc_score(y_val, y_pred))

np.mean(scores)

Learning rate set to 0.083772
0:	test: 0.5311027	best: 0.5311027 (0)	total: 136ms	remaining: 2m 16s
100:	test: 0.6736508	best: 0.6759926 (72)	total: 13.4s	remaining: 1m 59s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6759926368
bestIteration = 72

Shrink model to first 73 iterations.
Learning rate set to 0.083772
0:	test: 0.5591225	best: 0.5591225 (0)	total: 144ms	remaining: 2m 23s
100:	test: 0.6888262	best: 0.6907125 (76)	total: 13.5s	remaining: 2m
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6907125105
bestIteration = 76

Shrink model to first 77 iterations.
Learning rate set to 0.083772
0:	test: 0.5512153	best: 0.5512153 (0)	total: 53.6ms	remaining: 53.6s
100:	test: 0.6781938	best: 0.6804709 (35)	total: 13.3s	remaining: 1m 58s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6804709417
bestIteration = 35

Shrink model to first 36 iterations.
Learning rate set to 0.083772
0:	test: 0.5526005	best: 0.5526005 (0)	total

0.6791523958051826

In [10]:
model.get_feature_importance(prettified=True)[:30]

,Feature Id,Importances
0,employee_count_nm,23.786707
1,customer_age,17.057399
2,report,2.077658
3,embed_31,1.332290
4,embed_17,1.141502
5,embed_0,1.137816
6,embed_68,1.107971
7,embed_90,1.085583
8,embed_26,1.072870
9,embed_12,1.032052


## Собираем эмбеддинги в одну колонку

In [7]:
train_emb = pd.DataFrame(np.arange(len(ptls)))
train_emb['embeddings'] = ptls.drop(['user_id'], axis=1).values.tolist()
train_emb = train_emb.drop([0], axis=1)
train_emb['user_id'] = ptls.user_id
train_emb

,embeddings,user_id
0,"[-0.16553426, 0.35197815, -0.35083854, 0.10683...",3
1,"[-0.06302169, 0.3614778, -0.82655585, -0.15408...",9
2,"[-0.24683955, 0.10781428, 0.34188807, 0.161510...",13
3,"[-0.14562747, 0.02570638, -0.542809, -0.197881...",37
4,"[-0.08313994, 0.015028149, -0.032059304, 0.202...",41
...,...,...
95995,"[0.47807398, 0.80386764, -0.4670445, 0.1319316...",562043
95996,"[0.11833923, 0.19016266, -0.24537548, 0.118227...",562205
95997,"[-0.11397964, 0.17562571, -0.47836116, -0.1316...",562312
95998,"[-0.22644511, 0.4476551, -0.76360726, -0.13265...",562721


In [12]:
train_emb = train.merge(train_emb, on='user_id', how='left').fillna('nan')
train_emb

,user_id,target,time,report,employee_count_nm,bankemplstatus,customer_age,embeddings
0,3,0,77.0,2,ОТ 101 ДО 500,0,3,"[-0.16553426, 0.35197815, -0.35083854, 0.10683..."
1,13,0,86.0,6,ОТ 501 ДО 1000,0,2,"[-0.24683955, 0.10781428, 0.34188807, 0.161510..."
2,37,0,89.0,5,БОЛЕЕ 1001,0,2,"[-0.14562747, 0.02570638, -0.542809, -0.197881..."
3,41,0,57.0,1,ОТ 101 ДО 500,0,2,"[-0.08313994, 0.015028149, -0.032059304, 0.202..."
4,42,0,84.0,12,ДО 10,0,3,"[-0.16860683, -0.026093781, 0.15359479, -0.510..."
...,...,...,...,...,...,...,...,...
95995,561362,-1,nan,12,nan,0,3,"[0.058122456, 0.27963477, -0.6540566, -0.01714..."
95996,561419,-1,nan,12,nan,0,3,"[-0.15197878, 0.3140609, -0.55599904, -0.04182..."
95997,561895,-1,nan,12,nan,0,2,"[0.12500264, 0.62248385, -0.43214202, 0.084706..."
95998,561908,-1,nan,12,nan,0,2,"[0.15718761, 0.42154166, -0.23989606, 0.152016..."


In [13]:
X = train_emb[train_emb.target != -1].drop(['user_id', 'target', 'time'], axis=1)
y = train_emb[train_emb.target != -1]['target']                        

In [14]:
n_splits=5
scores = []
models = []

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for train_index, val_index in skf.split(X, y):
    
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    

    model = CatBoostClassifier(random_seed=42,
                               #auto_class_weights='Balanced',
                                eval_metric='AUC',
                                cat_features=cat_cols,
                                embedding_features=['embeddings'],
                                thread_count=-1,
                                #task_type='GPU',
    )

    model.fit(X_train, y_train,
              eval_set=(X_val, y_val),
              early_stopping_rounds=100,
              verbose=100)
    
    models.append(model)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    scores.append(roc_auc_score(y_val, y_pred))

np.mean(scores)

Learning rate set to 0.083772
0:	test: 0.5660609	best: 0.5660609 (0)	total: 106ms	remaining: 1m 46s
100:	test: 0.6830497	best: 0.6850393 (39)	total: 8.49s	remaining: 1m 15s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6850392817
bestIteration = 39

Shrink model to first 40 iterations.
Learning rate set to 0.083772
0:	test: 0.4838898	best: 0.4838898 (0)	total: 21.9ms	remaining: 21.8s
100:	test: 0.6925468	best: 0.6938091 (59)	total: 8s	remaining: 1m 11s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6938091231
bestIteration = 59

Shrink model to first 60 iterations.
Learning rate set to 0.083772
0:	test: 0.5459115	best: 0.5459115 (0)	total: 39.2ms	remaining: 39.2s
100:	test: 0.6840870	best: 0.6844187 (61)	total: 8.32s	remaining: 1m 14s
200:	test: 0.6839717	best: 0.6845240 (144)	total: 17.6s	remaining: 1m 9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6845240136
bestIteration = 144

Shrink model to first 145 iterations

0.6837971085956969

In [15]:
model.get_feature_importance(prettified=True)[:30]

,Feature Id,Importances
0,employee_count_nm,43.606951
1,customer_age,34.227475
2,report,11.327315
3,embeddings,6.781745
4,bankemplstatus,4.056513


## Создаём файл для отправки cабмита

In [16]:
test = pd.read_csv('sample_submit_naive.csv')
test['predict'] = 0
for i in range(len(models)):
    test['predict'] += models[i].predict_proba(train_emb[train_emb.target == -1].drop(['user_id', 'target', 'time'], axis=1))[:, 1]
test['predict'] = test['predict'] / 5
test.to_csv('submission.csv', index=False)
test

,user_id,predict
0,9,0.043031
1,61,0.055646
2,62,0.060504
3,80,0.039632
4,88,0.114543
...,...,...
31995,561362,0.081646
31996,561419,0.083867
31997,561895,0.086813
31998,561908,0.086215
